<a href="https://colab.research.google.com/github/Rafael-Maranhao/portfolio/blob/main/notebook_startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🌱 Levantamento de Dados: Filial de Startup Agrícola
<br>

#### Objetivo

<br>

##### Identificar as melhores cidades para instalação de um novo escritório, considerando:
<br>

##### 🏙️ Top 100 cidades mais populosas do Brasil;


##### 🌾 Ter áreas produtoras de soja.

<br>
<br>

### 📚 Importação de Bibliotecas

In [58]:
import urllib3
import json
import pandas as pd
import os

### 🔍 Visualização Completa (Pandas)  

In [59]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 📡 Importação de dados da API do IBGE

In [60]:
# Função para obter dados da API com tratamento de erro
def get_data(url):
    http = urllib3.PoolManager()
    try:
        r = http.request('GET', url)
        return json.loads(r.data)
    except Exception as e:
        print(f"Erro ao acessar API: {str(e)}")
        return None

# Obter dados
print("Coletando dados de população...")
dados_populacao = get_data('https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[all]')

print("Coletando dados de área de soja...")
dados_soja = get_data('https://servicodados.ibge.gov.br/api/v3/agregados/1612/periodos/2021/variaveis/109?localidades=N6[all]&classificacao=81[2713]')

Coletando dados de população...
Coletando dados de área de soja...


### 📊 Processamento dos dados (criação de dataframes para os dados de população e de área de soja)

In [61]:
# Função para processar dados de população
def processar_populacao(dados):
    municipios = []
    if dados and len(dados) > 0:
        for item in dados[0]['resultados'][0]['series']:
            try:
                municipios.append({
                    'codigo': item['localidade']['id'],
                    'municipio': item['localidade']['nome'],
                    'populacao': int(item['serie']['2021'])
                })
            except:
                continue
    return pd.DataFrame(municipios)

# Função para processar dados de soja
def processar_soja(dados):
    municipios = []
    if dados and len(dados) > 0:
        for item in dados[0]['resultados'][0]['series']:
            try:
                valor = item['serie']['2021']
                # Considera apenas valores numéricos positivos
                if valor not in ['-', '...', '0']:
                    area = float(valor)
                    if area > 0:
                        municipios.append({
                            'codigo': item['localidade']['id'],
                            'municipio': item['localidade']['nome'],
                            'area_soja': area
                        })
            except:
                continue
    return pd.DataFrame(municipios)

# Processar os dados
print("Processando dados...")
df_pop = processar_populacao(dados_populacao)
df_soja = processar_soja(dados_soja)

Processando dados...


### 🏙️ Filtragem dos 100 Municípios Mais Populosos

In [62]:
top_100_pop = df_pop.nlargest(100, 'populacao')

### 🔄 Cruzamento de Dados (de população com de áreas de soja)

In [63]:
df_final = pd.merge(top_100_pop, df_soja, on=['codigo', 'municipio'], how='inner')

### 📊 Ordenação Final

In [64]:
df_final = df_final.sort_values('populacao', ascending=False)

### ✨ Formatação dos Resultados

In [65]:
df_final['area_soja'] = df_final['area_soja'].apply(lambda x: f"{x:,.0f} ha")
df_final['populacao'] = df_final['populacao'].apply(lambda x: f"{x:,}")

### 🎯 Resultados Finais

In [66]:
print("\n" + "="*80)
print("MUNICÍPIOS RECOMENDADOS PARA O NOVO ESCRITÓRIO".center(80))
print("(Entre os 100 mais populosos com produção de soja > 0)".center(80))
print("="*80 + "\n")

# Configurar a formatação da tabela
from tabulate import tabulate

# Formatar os dados para exibição
df_display = df_final[['municipio', 'populacao', 'area_soja']].copy()
df_display.columns = ['MUNICÍPIO', 'POPULAÇÃO', 'ÁREA DE SOJA (ha)']  # Renomear colunas

# Usar tabulate para melhor formatação
print(tabulate(df_display,
               headers='keys',
               tablefmt='pretty',
               stralign='left',
               showindex=False,
               colalign=('left', 'right', 'right')))

print("\n" + "="*80)
print(f"Total de municípios recomendados: {len(df_final)}".center(80))
print("="*80)


                 MUNICÍPIOS RECOMENDADOS PARA O NOVO ESCRITÓRIO                 
             (Entre os 100 mais populosos com produção de soja > 0)             

+----------------------------+-----------+-------------------+
| MUNICÍPIO                  | POPULAÇÃO | ÁREA DE SOJA (ha) |
+----------------------------+-----------+-------------------+
| Brasília - DF              | 3,094,325 |         80,000 ha |
| Goiânia - GO               | 1,555,626 |          1,150 ha |
| Porto Alegre - RS          | 1,492,530 |            398 ha |
| Campinas - SP              | 1,223,237 |            800 ha |
| Campo Grande - MS          |   916,001 |         94,000 ha |
| Ribeirão Preto - SP        |   720,116 |            700 ha |
| Uberlândia - MG            |   706,597 |         58,000 ha |
| Sorocaba - SP              |   695,328 |            700 ha |
| Cuiabá - MT                |   623,614 |            338 ha |
| Londrina - PR              |   580,870 |         62,000 ha |
| Porto Velho - R

### 💾 Exportação dos Resultados

In [67]:
# Salvar em CSV (mantendo o original)
df_final.to_csv('municipios_recomendados.csv', index=False)
print("\nArquivo 'municipios_recomendados.csv' salvo com todos os dados.")


Arquivo 'municipios_recomendados.csv' salvo com todos os dados.
